In [2]:
!pip install -q datasets
!pip install -q -U bitsandbytes
!pip install -q -U flash_attn
!pip install -q evaluate
!pip install -q rouge_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 7.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.

In [17]:
!pip install -q sentence-transformers

In [1]:
import torch
from datasets import Dataset, load_dataset
from transformers import pipeline, BitsAndBytesConfig
import bitsandbytes as bnb
import random
import evaluate
import pandas as pd
from tqdm import tqdm
from pprint import pprint
from sentence_transformers import SentenceTransformer, util

/opt/cortex-installs/miniconda/envs/w266/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# This cell will authenticate you and mount your Drive in the Colab.
# from google.colab import drive
# drive.mount('/content/drive')
# llama_dir = 'drive/MyDrive/datasci266 Natural Language Processing/Project/llama_output/'
llama_dir = './llama_output/'

In [3]:
# Set random seed for reproducibility
random.seed(42)
torch.manual_seed(42)

ds = load_dataset("rony/climate-change-MRC")

In [4]:
def prepare_data(data):
  articles = []

  for paragraph in data:
    context = paragraph['context']
    for qa in paragraph['qas']:
      question = qa['question']
      id = qa['id']
      for ans in qa['answers']:
        answer = ans['text']
        answer_start = ans['answer_start']
        articles.append({'context': context, 'question': question, 'id': id, 'answer': answer, 'answer_start': answer_start})

  return articles

In [5]:
train_ds = ds["train"]
valid_ds = ds["validation"]
test_ds = ds["test"]

# each is a 1-item list, so take first index
train_ds = train_ds[0]
valid_ds = valid_ds[0]
test_ds = test_ds[0]

# take the 'data' key of the dict, ignoring 'version' (there's just one)
train_ds = train_ds['data'][0]['paragraphs']
valid_ds = valid_ds['data'][0]['paragraphs']
test_ds = test_ds['data'][0]['paragraphs']
# each dataset is a list of dicts, where each list item is a context paragraph ('context' key) with qas ('qas' key) which contain questions, id, and answer

train_df = pd.DataFrame(prepare_data(train_ds))
print(f"{train_df.shape=}")

valid_df = pd.DataFrame(prepare_data(valid_ds))
print(f"{valid_df.shape=}")

test_df = pd.DataFrame(prepare_data(test_ds))
print(f"{test_df.shape=}")

train_df.shape=(14756, 5)
valid_df.shape=(4229, 5)
test_df.shape=(2096, 5)


In [6]:
train_df.head()

,context,question,id,answer,answer_start
0,"outside of the clean air act, there is support...",Is a mandate for electric production to come f...,665,"and there is ongoing interest in a ""national r...",222
1,"outside of the clean air act, there is support...",How wide ranging are climate policies across t...,666,"as of 2010, climate policies were being contem...",618
2,"outside of the clean air act, there is support...",What kind of energy standards have been implem...,667,"at latest count, 30 states have implemented re...",734
3,acknowledgments. we thank jim haywood for the ...,What was Jim Haywood thanked for?,1731,we thank jim haywood for the aerosol forcing pdf,17
4,acknowledgments. we thank jim haywood for the ...,Where was Jonathan Gregory supported?,1732,jonathan gregory was supported at the universi...,277


In [ ]:
# !huggingface-cli login

In [11]:
"""
Initialize the pipeline with bitsandbytes quantization
"""
# Configure bitsandbytes for 4-bit quantization
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Initialize pipeline
model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

summarizer = pipeline(
   "text-generation",
   model=model_id,
   model_kwargs={"torch_dtype": torch.bfloat16, "quantization_config": quantization_config},
   device_map="auto",
   trust_remote_code=True,
   batch_size=8,
  #  use_auth_token=True
)

Loading checkpoint shards: 100%|██████████| 4/4 [02:01<00:00, 30.35s/it]
Device set to use cuda:0


In [ ]:
prompt = [
            {"role": "system", "content": "You are skilled in answering questions."},
            {"role": "user", "content": "Generate an answer to this question using the provided context. The format will be the context followed by the question: "},
            {"role": "user", "content": test_df['context'][0]},
            {"role": "user", "content": test_df['question'][0]}
        ]


outputs = summarizer(
  prompt,
  max_new_tokens=512,
  do_sample = True,
  temperature = 0.3,
  top_p = 0.95,
)

answer = outputs[0]["generated_text"][-1]

In [ ]:
answer

In [ ]:
test_df['answer'][0]

In [ ]:
rouge = evaluate.load('rouge')

# Calculate ROUGE scores
predictions = [answer['content']]
references = [test_df['answer'][0]]
rouge_scores = rouge.compute(predictions=predictions, references=references)
rouge_scores

In [6]:
rouge = evaluate.load('rouge')

In [11]:
# experiment with parameters on a sample of rows
random.seed(888)
sample_indices = random.sample(range(len(train_df)), 50)
sample_df = train_df.iloc[sample_indices]

In [28]:
results = []
for idx, sample in tqdm(sample_df.iterrows(),total=sample_df.shape[0]):
    try:
      prompt = [
            {"role": "system", "content": "You are skilled in answering questions."},
            {"role": "user", "content": "Generate an answer to this question using the provided context. The format will be the context followed by the question: "},
            {"role": "user", "content": test_df['context'][idx]},
            {"role": "user", "content": test_df['question'][idx]}
      ]

      # Generate summary via the pipeline
      outputs = summarizer(
                          prompt,
                          max_new_tokens=512,
                          do_sample = True,
                          pad_token_id = summarizer.tokenizer.eos_token_id,
                          temperature = 0.5,
                          top_p = 0.4
      )

      answer = outputs[0]["generated_text"][-1]

      # Calculate ROUGE scores
      predictions = [answer['content']]
      references = [[sample['answer']]]
      rouge_scores = rouge.compute(predictions=predictions, references=references)


      # Store results
      results.append({
          'id': idx,
          'original_context': sample['context'],
          'original_question': sample['question'],
          'reference_answer': sample['answer'],
          'generated_answer': answer,
           **rouge_scores
      })

      # Print progress update every 200 samples
      if (idx + 1) % 200 == 0:
          print(f"\nProcessed {idx + 1} samples")
          print(f"Latest ROUGE-1: {rouge_scores['rouge1']:.4f}")

    except Exception as e:
      print(f"Error processing sample {idx}: {str(e)}")
      continue

100%|██████████| 50/50 [07:52<00:00,  9.44s/it]


In [29]:
print(rouge_scores)

predictions = [results[i]['generated_answer'] for i in range(len(results))]
references = [results[i]['reference_answer'] for i in range(len(results))]
encoder_model = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2')
candidate_embeddings = encoder_model.encode(predictions)
reference_embeddings = encoder_model.encode(references)
similarity = util.pairwise_cos_sim(candidate_embeddings, reference_embeddings)
print(f"similarity: {torch.mean(similarity)}")

{'rouge1': np.float64(0.4999999999999999), 'rouge2': np.float64(0.42857142857142855), 'rougeL': np.float64(0.44000000000000006), 'rougeLsum': np.float64(0.44000000000000006)}
similarity: 0.024155138060450554


| max_new_tokens | do_sample | temperature | top_p | rouge-1             | rouge-2  | rouge-L | semantic similarity
|----------------|-----------|-------------|-------|---------------------|--------------------|---------------------|--------|
| 512  | True | 0.3  | 0.95  | 0.47826086956521735 | 0.4444444444444445 | 0.47826086956521735 | |
| 512| True  |  1.0  |  0.95  |  0.24166666666666664       | 0.13445378151260506  |   0.2                | 0.024155138060450554|
|512  | True   |  1.0  |    0.3   | 0.4999999999999999 | 0.42857142857142855 | 0.44000000000000006 | 0.024155138060450554|
| 512 | True | 0.5 | 0.3 |0.46808510638297873 |0.4347826086956522 | 0.46808510638297873|0.024155138060450554|
| 512 | True | 0.5 | 0.4| 0.4999999999999999|0.42857142857142855 |0.44000000000000006 | 0.024155138060450554|

In [30]:
pprint(results[0])

{'generated_answer': {'content': 'According to the text, the main factors that '
                                 'contributed to the reduction of sediment '
                                 'delivery to the estuary by about half were:\n'
                                 '\n'
                                 '1. Cessation of hydraulic mining\n'
                                 '2. Flood management\n'
                                 '3. Damming the large rivers',
                      'role': 'assistant'},
 'id': 327,
 'original_context': 'our projections illustrate how responses to climate '
                     'change could transform the sfew into a very different '
                     'system by midcentury (fig. 2). transformative change is '
                     'not new to this ecosystem, which has been altered over '
                     'the past 150 years by massive landscape modifications, '
                     'water development, pollutant inputs and s of alien '
         

In [12]:
test_df = pd.read_csv('test_df_tfidf_retrieval.csv')
test_df.tail(2)

,context,question,id,answer,answer_start,retrieved_context,retrieval_correct
2094,"zealand, but not explicit policies to accept p...",How are fieldwork interviews conducted?,6283,interviews from fieldwork revealed mixed views...,113,"zealand, but not explicit policies to accept p...",True
2095,"zealand, but not explicit policies to accept p...",Climate Say sea level rise?,6284,some believe that climate negotiations that se...,392,we conclude that pliocene temperatures probabl...,False


In [13]:
results = []
batch_results = []
csv_path = llama_dir+'results_w_retrieval.csv'

# Ensure the CSV has headers only once
header_written = False

for idx, sample in tqdm(test_df.iterrows(), total=test_df.shape[0]):
    try:
        prompt = [
            {"role": "system", "content": "You are skilled in answering questions."},
            {"role": "user", "content": "Generate an answer to this question using the provided context. The format will be the context followed by the question: "},
            {"role": "user", "content": test_df['retrieved_context'][idx]},
            {"role": "user", "content": test_df['question'][idx]}
        ]

        outputs = summarizer(
            prompt,
            max_new_tokens=512,
            do_sample=True,
            pad_token_id=summarizer.tokenizer.eos_token_id,
            temperature=1,
            top_p=0.3
        )

        answer = outputs[0]["generated_text"][-1]

        predictions = [answer['content']]
        references = [[sample['answer']]]
        rouge_scores = rouge.compute(predictions=predictions, references=references)

        result = {
            'id': idx,
            'original_context': sample['context'],
            'retrieved_context': sample['retrieved_context'],
            'original_question': sample['question'],
            'reference_answer': sample['answer'],
            'generated_answer': answer,
            **rouge_scores
        }

        results.append(result)
        batch_results.append(result)

        # Write every 50 results to CSV
        if len(batch_results) >= 50:
            df_batch = pd.DataFrame(batch_results)
            df_batch.to_csv(csv_path, mode='a', header=not header_written, index=False)
            batch_results = []
            header_written = True

        # Print progress every 200 samples
        if (idx + 1) % 200 == 0:
            print(f"\nProcessed {idx + 1} samples")
            print(f"Latest ROUGE-1: {rouge_scores['rouge1']:.4f}")

    except Exception as e:
        print(f"Error processing sample {idx}: {str(e)}")
        continue

# Save any remaining results
if batch_results:
    df_batch = pd.DataFrame(batch_results)
    df_batch.to_csv(csv_path, mode='a', header=not header_written, index=False)


 10%|▉         | 200/2096 [30:45<7:22:45, 14.01s/it] 


Processed 200 samples
Latest ROUGE-1: 0.0733


 19%|█▉        | 400/2096 [1:05:17<3:32:36,  7.52s/it]


Processed 400 samples
Latest ROUGE-1: 0.9455


 29%|██▊       | 600/2096 [1:34:10<2:41:17,  6.47s/it] 

...

 86%|████████▌ | 1800/2096 [4:23:57<1:00:06, 12.18s/it]


Processed 1800 samples
Latest ROUGE-1: 0.8756


 95%|█████████▌| 2000/2096 [4:58:12<18:04, 11.29s/it]  


Processed 2000 samples
Latest ROUGE-1: 0.0800


100%|██████████| 2096/2096 [5:11:33<00:00,  8.92s/it]


In [14]:
# print(rouge_scores)
predictions = [results[i]['generated_answer']['content'] for i in range(len(results))]
references = [results[i]['reference_answer'] for i in range(len(results))]
rouge_calculated = rouge.compute(predictions=predictions, references=references)
print(rouge_calculated)

encoder_model = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2')
candidate_embeddings = encoder_model.encode(predictions)
reference_embeddings = encoder_model.encode(references)
similarity = util.pairwise_cos_sim(candidate_embeddings, reference_embeddings)
print(f"similarity: {torch.mean(similarity)}")

{'rouge1': np.float64(0.4238395716024037), 'rouge2': np.float64(0.32620324593972716), 'rougeL': np.float64(0.3871348139339277), 'rougeLsum': np.float64(0.3919156961468551)}
similarity: 0.669610321521759


In [ ]:
for idx, sample in tqdm(test_df.iterrows(), total=test_df.shape[0]):
    try:
      prompt = [
            {"role": "system", "content": "You are skilled in answering questions."},
            {"role": "user", "content": "Generate an answer to this question using the provided context. The format will be the context followed by the question: "},
            {"role": "user", "content": test_df['context'][idx]},
            {"role": "user", "content": test_df['question'][idx]}
      ]

      # Generate summary via the pipeline
      outputs = summarizer(
                          prompt,
                          max_new_tokens=512,
                          do_sample = True,
                          pad_token_id = summarizer.tokenizer.eos_token_id,
                          temperature = 0.3,
                          top_p = 0.95
      )

      answer = outputs[0]["generated_text"][-1]

      # Calculate ROUGE scores
      predictions = [answer['content']]
      references = [[sample['answer']]]
      rouge_scores = rouge.compute(predictions=predictions, references=references)


      # Store results
      results.append({
          'id': idx,
          'original_context': sample['context'][:500],
          'original_question': sample['question'],
          'reference_answer': sample['answer'],
          'generated_answer': answer,
           **rouge_scores
      })

      # Print progress update every 200 samples
      if (idx + 1) % 200 == 0:
          print(f"\nProcessed {idx + 1} samples")
          print(f"Latest ROUGE-1: {rouge_scores['rouge1']:.4f}")

    except Exception as e:
      print(f"Error processing sample {idx}: {str(e)}")
      continue

200it [31:31, 10.29s/it]


Processed 200 samples
Latest ROUGE-1: 0.5357


400it [1:06:16, 10.84s/it]


Processed 400 samples
Latest ROUGE-1: 0.9455


502it [1:20:51,  9.66s/it]


KeyboardInterrupt: 

In [ ]:
# results for first 10 iterations
# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Calculate and print average ROUGE scores
avg_scores = results_df[['rouge1', 'rouge2', 'rougeL']].mean()
print("\nAverage ROUGE Scores:")
for metric, score in avg_scores.items():
   print(f"{metric}: {score:.4f}")

# Print some example answers
print("\nExample Summaries:")
for i in range(min(5, len(results_df))):
   print(f"\nExample {i+1}:")
   print(f"Reference: {results_df.iloc[i]['reference_answer']}")
   print(f"Generated: {results_df.iloc[i]['generated_answer']}")


Average ROUGE Scores:
rouge1: 0.5633
rouge2: 0.4852
rougeL: 0.5332

Example Summaries:

Example 1:
Reference: flooding damage and climate variability
Generated: {'role': 'assistant', 'content': 'According to the text, the 9 percent reduction of rice production in Bangladesh is attributed to two variables:\n\n1. Flooding damage\n2. Climate variability'}

Example 2:
Reference: a dynamic economywide model
Generated: {'role': 'assistant', 'content': 'A dynamic economy-wide model of Bangladesh was used to estimate economic damages from historical climate variability and future anthropogenic climate change.'}

Example 3:
Reference: a modelling approach
Generated: {'role': 'assistant', 'content': 'According to the text, Ahmed et al. (2011) used a modelling approach to estimate how changes in climate variability might affect crop yields and thence poverty rates in Tanzania to the early 2030s.'}

Example 4:
Reference: extreme sea level height fluctuations are also larger to the north, as a res

In [ ]:
# results for 502 iterations
# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Calculate and print average ROUGE scores
avg_scores = results_df[['rouge1', 'rouge2', 'rougeL']].mean()
print("\nAverage ROUGE Scores:")
for metric, score in avg_scores.items():
   print(f"{metric}: {score:.4f}")

# Print some example answers
print("\nExample Summaries:")
for i in range(200, 210):
   print(f"\nExample {i+1}:")
   print(f"Reference: {results_df.iloc[i]['reference_answer']}")
   print(f"Generated: {results_df.iloc[i]['generated_answer']}")


Average ROUGE Scores:
rouge1: 0.5096
rouge2: 0.4167
rougeL: 0.4696

Example Summaries:

Example 201:
Reference: models and empirical studies suggest that the speed with which a population adapts to a changing environment affects invasion rate of new habitat and thus migration rate, population growth rate and thus probability of extinction, and growth and mortality of individual plants and thus productivity of regional vegetation
Generated: {'role': 'assistant', 'content': 'According to the text, the models and empirical studies revealed that the speed with which a population adapts to a changing environment affects several key factors, including:\n\n1. Invasion rate of new habitat\n2. Migration rate\n3. Population growth rate and thus probability of extinction\n4. Growth and mortality of individual plants and thus productivity of regional vegetation.'}

Example 202:
Reference: as this study covers a large spatial scale, we anticipate that results are also valid for other regions
Gener